In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 46.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
#load pdf files
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/ragfty/sample_data")
data = loader.load()

In [ ]:
print(data)

[Document(page_content=' \nAmazon Kindle \nPublishing Guidelines  \nHow to make books available for Kindle \ndevices and applications  \n \n \nversion 2019.2  \n \n \n \n \n \n \nThis document describes the primary ways publishers, authors, and conversion houses can \nmake their content available for Amazon Kindle devices and applications . This document \nincludes guidelines and suggestions to ensure a smooth conversion and publication process.  \n \nCopyright © 201 9 Amazon.com, Inc. or its affiliates. All rights reserved.  \n  \n', metadata={'source': '/content/drive/MyDrive/ragfty/sample_data/AmazonKindlePublishingGuidelines.pdf', 'page': 0}), Document(page_content="Publishing on Kindle: Guidelines for Publishers  \nKindle Publishing Guidelines  Amazon.com  2 Revision History  \nRevision \nNumber  Revision Notes  \n2019.2  \uf0b7 Updated the following sections to add accessibility guidelines:  \no 2.1.1 Kindle Create  \no 3 Comparing Formats  \no 6.1 Constructing Well -Formed HTML 

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

126

In [ ]:
#get the third chunk
text_chunks[2]

Document(page_content="Publishing on Kindle: Guidelines for Publishers  \nKindle Publishing Guidelines  Amazon.com  3 Table of Contents  \nPART I. GETTING STAR TED  ................................ ................................ ................................ ... 8 \n1 Introd uction  ................................ ................................ ................................ ................................ .....................  8 \n2 Paths to Getting Your Content on Kindle ................................ ................................ ................................ ... 8 \n Amazon’s Kindle Direct Publishing Platform  ................................ ................................ ................................  8 \n2.1.1 Kindle Create  ................................ ................................ ................................ ................................ .............  8 \n Creating K indle Books In -House Using Kindle Publisher Tools  ..................

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
#connect to google drive
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/ragfty/model/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/ragfty/model/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: 

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [ ]:
query = "what is documentation cost"

In [ ]:
qa.run(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =    3631.14 ms
llama_print_timings:      sample time =      23.19 ms /    42 runs   (    0.55 ms per token,  1811.13 tokens per second)
llama_print_timings: prompt eval time =  636340.44 ms /  1283 tokens (  495.98 ms per token,     2.02 tokens per second)
llama_print_timings:        eval time =   28283.45 ms /    41 runs   (  689.84 ms per token,     1.45 tokens per second)
llama_print_timings:       total time =  665408.59 ms /  1324 tokens


' Documentation costs money so it must be included in your event budget. The cost depends on the methods chosen. It is a good idea to get quotes from professionals and look for ways to make it cheaper.'

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3631.14 ms
llama_print_timings:      sample time =      60.18 ms /   106 runs   (    0.57 ms per token,  1761.47 tokens per second)
llama_print_timings: prompt eval time =  341896.22 ms /   677 tokens (  505.02 ms per token,     1.98 tokens per second)
llama_print_timings:        eval time =   72747.25 ms /   105 runs   (  692.83 ms per token,     1.44 tokens per second)
llama_print_timings:       total time =  415363.81 ms /   782 tokens


Answer:  Document cost refers to the money that is spent on creating or maintaining documentation, which includes tasks such as documentation management, knowledge discovery, capturing and creation, and knowledge classification and representation. The cost of documentation depends on the methods chosen and can vary greatly depending on factors such as the level of detail and complexity required for the documentation, as well as the expertise and resources available for creating and maintaining it. It is important to consider the cost of documentation when planning events or projects and to seek out cost-effective solutions whenever possible.
